# Задание 5

## 1. ContextTimer(0.3 балла)
Напишите менеджер контекста, который позволит засекать время выполнения блока кода с помощью конструкции with и выводить это время на экран по выходу из блока. Пример использования:

<code>
with Timer ():
    do_some_long_stuff()
</code>

Рекомендации: используйте стандартную библиотеку для работы со временем.

Протестируйте себя.

In [1]:
import time
import datetime

class Timer():
    def __init__(self):
        self.start = 0
        self.end = 0
        
    def __enter__(self):
        self.start = time.time()
        
    def __exit__(self, *args):
        self.end = time.time()
        self.delta = self.end - self.start
        print(str(datetime.timedelta(seconds=self.delta)))

In [17]:
import sys
sys.setrecursionlimit(10**5)

In [25]:
def fact(n):
    if (n < 2):
        return 1
    return (n * fact(n - 1)) 

In [26]:
with Timer():
    fact(5000)

0:00:00.009794


In [27]:
%%time
fact(1000)

CPU times: user 926 µs, sys: 15 µs, total: 941 µs
Wall time: 950 µs


4023872600770937735437024339230039857193748642107146325437999104299385123986290205920442084869694048004799886101971960586316668729948085589013238296699445909974245040870737599188236277271887325197795059509952761208749754624970436014182780946464962910563938874378864873371191810458257836478499770124766328898359557354325131853239584630755574091142624174743493475534286465766116677973966688202912073791438537195882498081268678383745597317461360853795345242215865932019280908782973084313928444032812315586110369768013573042161687476096758713483120254785893207671691324484262361314125087802080002616831510273418279777047846358681701643650241536913982812648102130927612448963599287051149649754199093422215668325720808213331861168115536158365469840467089756029009505376164758477284218896796462449451607653534081989013854424879849599533191017233555566021394503997362807501378376153071277619268490343526252000158885351473316117021039681759215109077880193931781141945452572238655414610628921879602238389714760

## 2.Transaction (0.4 балла)
Вам необходимо написать менеджер контекстов, который позволит безопасно работать с транзакциями. Напишите класс Storage, в котором будут храниться какие-то данные в виде словаря. Эти данные должны быть закрытыми и их можно только читать через операцию []. У этого класса должен быть метод edit, который возвращает менеджер контекста, позволяющий редактировать исходный объект (опять же через []). При этом результаты редактирования записываются в исходный объект только если весь блок выполнился успешно. Пример использования:

<code>
s = Storage()
with s.edit() as se :
    se["a"] = 1
    may_be_an_exception_here()
</code>

Протестируйте себя.

In [28]:
import copy
class Storage:
    class Manager():
        def __init__(self, dict_base):
            self.__manager_dict = copy.deepcopy(dict_base)
            self.dict = dict_base
        
        def __enter__(self):
            return self
               
        def __exit__(self, *args):
            for arg in args:
                if arg is not None:
                    self.__manager_dict = self.dict
                    pass
            self.dict = copy.deepcopy(self.__manager_dict)
            
        def __setitem__(self, key, value):
            self.__manager_dict[key] = value
        
    def __init__(self, base_dict={}):
        self.__manager = Storage.Manager(base_dict)
        
    def __getitem__(self, key):
        if key in self.__manager.dict:
            return self.__manager.dict[key]
        
    def edit(self):
        return self.__manager

In [29]:
s = Storage({})

with s.edit() as se:
    se["a"] = 10
try:
    with s.edit() as se:
        se["b"] = 20
        raise Exception()
except:
    pass

print(s["a"])
print(s["b"])

10
None


In [34]:
s = Storage({1: 'a', 2: 'b'})
try:
    with s.edit() as se:
        se[2] = 'c'
        raise Exception()
except:
    print("exception caught")
    pass

exception caught


In [31]:
print(s[1], s[2])


a b


In [32]:
with s.edit() as se:
    se[1] = 'c'
print(s[1], s[2])

c b


## Phone numbers (0.3)
Напишите регулярное выражение для распознавания телефонных номеров и протестируйте себя. Номера, которые должны распознаваться:
* 3-22-46
* +71239513749
* 71239513749
* 1239513749
* +7(123)-951-37-49
* +7(123)9513749
* +7-123-9513749
* +7-123-951-37-49

* 7(123)-951-37-49
* 7(123)9513749
* 7-123-9513749
* 7-123-951-37-49

* (123)-951-37-49
* (123)9513749
* 123-9513749
* 123-951-37-49

In [90]:
import re
str_sample = """3-22-46
* +71239513749
* 71239513749
* 1239513749
* +7(123)-951-37-49
* +7(123)9513749
* +7-123-9513749
* +7-123-951-37-49
* 7(123)-951-37-49
* 7(123)9513749
* 7-123-9513749
* 7-123-951-37-49
* (123)-951-37-49
* (123)9513749
* 123-9513749
* 123-951-37-49
* +7(999 999-99-99
* +7-999 999-99-99
* +7(999)999-99-99
* +7(999)999-99-999"""

In [91]:
samples = str_sample.split('\n* ')
samples

['3-22-46',
 '+71239513749',
 '71239513749',
 '1239513749',
 '+7(123)-951-37-49',
 '+7(123)9513749',
 '+7-123-9513749',
 '+7-123-951-37-49',
 '7(123)-951-37-49',
 '7(123)9513749',
 '7-123-9513749',
 '7-123-951-37-49',
 '(123)-951-37-49',
 '(123)9513749',
 '123-9513749',
 '123-951-37-49',
 '+7(999 999-99-99',
 '+7-999 999-99-99',
 '+7(999)999-99-99',
 '+7(999)999-99-999']

In [92]:
#pattern = r'(\+\d+|\d*)(\-?\d+|\(?\d+\)?)(\-?\d+){1,3}'
pattern = r'(\+?\d)?\s*\-?\s*(\d{3}|\(\d{3}\))\s*\-?\s*(\d{3}|\(\d{3}\))\s*\-?\s*(\d{2}|\(\d{2}\))\s*\-?\s*(\d{2}|\(\d{2}\))'

res = re.match(pattern, '+7(123)-951-37-49')
res

<re.Match object; span=(0, 17), match='+7(123)-951-37-49'>

In [96]:

for sample in samples:
    m = re.match(pattern, sample)
    print(sample, m is not None and m.start() == 0 and m.end() == len(sample))

3-22-46 False
+71239513749 True
71239513749 True
1239513749 True
+7(123)-951-37-49 True
+7(123)9513749 True
+7-123-9513749 True
+7-123-951-37-49 True
7(123)-951-37-49 True
7(123)9513749 True
7-123-9513749 True
7-123-951-37-49 True
(123)-951-37-49 True
(123)9513749 True
123-9513749 True
123-951-37-49 True
+7(999 999-99-99 False
+7-999 999-99-99 True
+7(999)999-99-99 True
+7(999)999-99-999 False
